In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math as m
import os
import datetime
import pandas as pd
import import_ipynb

In [2]:
path_raw = '/Users/micol/Documents/Imperial/Year4/Master_Project/raw_spectra' #path with raw spectra files

In [3]:
file_array = []     #empty array to append filenames in original order (wrong order)
all_the_files = np.array([i for i in os.listdir(path_raw) if os.path.isfile(os.path.join(path_raw,i)) and i.startswith('IC_raw_spectra_')]) #all files in directory starting with 'IC_raw_spectra'

for f in all_the_files:
    file_array.append(f.strip('IC_raw_spectra.tx'))      #stripping file names to get the number at the end

file_integers = sorted([int(i) for i in file_array])     #number converted to sorted integers (not naturally done)


all_files_list = []    #final file list, in order

for num in file_integers: #add integer to string to make files_list
    all_files_list.append('IC_raw_spectra_'+str(num)+'.txt')

In [4]:
"""removing files without associated temperature measurement"""

def read_temperatures_check(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines() #finds all lines in file as string elements in "lines" list
    temp1 = [float(b) for b in (lines[1].split())[6:]]
    tempHBB1 = temp1[0]

    return tempHBB1

In [5]:
delete_files = [] #files where there is no temperature measurement

for f in all_files_list:
    if read_temperatures_check(path_raw, f) == 0.0:
        delete_files.append(f)

files_list = [x for x in all_files_list if x not in delete_files]

In [6]:
"""retrieving times from files"""

def read_times(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines() #finds all lines in file as string elements in "lines" list
    
    times = lines[0].split() #retrieve times of beginning of all spectra in a file
    return times

In [7]:
all_times_list = [] #final time list [374x4] (in HH:MM:SS.microsec format)

for f in range(len(files_list)): #go through files_list and extract times
    all_times_list.append(read_times(path_raw, files_list[f]))

flat_list = [item for sublist in all_times_list for item in sublist] #1D list of all times (in HH:MM:SS.microsec format)


spectrum_times = [] #final time list [(374*4)x1] in seconds from midnight format

for i in flat_list: #convert times to seconds format
    time = datetime.datetime.strptime(i, '%H:%M:%S.%f')
    spectrum_times.append((time.microsecond)/1000000 +time.second+time.minute*60+time.hour*3600)

In [8]:
time_col_name = 'Time'

In [9]:
# Read the log file - extract log_times and log_h_or_c
log_file = open(os.path.join('/Users/micol/Documents/Imperial/Year4/Master_Project/bruker_logs', 'EventLog_20180927112225.log')).readlines()

log_file_list = []
for i in log_file:
    log_file_list.append(i.split(' '))
del log_file_list[0:9]
for i in log_file_list:
    i.pop(5)
    i.pop(4)
    i.pop(3)
    i.pop(0)
# each item has length either 3 or 4. Length 4 is at start of h/c BB measurement

In [10]:
log_times = [] # all times in seconds since midnight
log_h_c = []

for i in log_file_list:
    time = datetime.datetime.strptime(i[0]+i[1], '%I:%M:%S%p')
    log_times.append(time.second+time.minute*60+time.hour*3600)
    if len(i) == 4:
        log_h_c.append(i[-1][0])
    elif len(i) == 3:
        log_h_c.append(log_h_c[-1])
    else:
        raise Exception('Error')

In [11]:
hc_log = pd.DataFrame(
    {time_col_name: log_times,
     'h/c': log_h_c,
    })
hc_log[400:450]

,Time,h/c
400,45516,c
401,45519,c
402,45521,c
403,45524,c
404,45526,c
405,45528,c
406,45530,c
407,45532,c
408,45536,c
409,45536,h


In [12]:
'''
times = spectrum_times # list of times in seconds since midnight

# make the two lists start and end at roughly same times

min_times = min(times)
min_hc = min(hc_log[time_col_name])

print(min_times)
print(min_hc)

if min_times <= min_hc:
    for t in times:
        if t < (min_hc):
            times.remove(t)
else:
    hc_log = hc_log[hc_log.Time >= min_times-3]
    
print(min(times))
print(min(hc_log[time_col_name]))

hc_log = hc_log.reset_index(drop=True)


max_times = max(times)
max_hc = max(hc_log[time_col_name])

if max_times > max_hc:
    for t in times:
        if t > (max_hc):
            times.remove(t)    
else:
    hc_log = hc_log[hc_log.Time <= max_times+3]            

hc_log = hc_log.reset_index(drop=True)  # reindex dataframe so that it starts from 0 after having removed rows in it 
'''



'\ntimes = spectrum_times # list of times in seconds since midnight\n\n# make the two lists start and end at roughly same times\n\nmin_times = min(times)\nmin_hc = min(hc_log[time_col_name])\n\nprint(min_times)\nprint(min_hc)\n\nif min_times <= min_hc:\n    for t in times:\n        if t < (min_hc):\n            times.remove(t)\nelse:\n    hc_log = hc_log[hc_log.Time >= min_times-3]\n    \nprint(min(times))\nprint(min(hc_log[time_col_name]))\n\nhc_log = hc_log.reset_index(drop=True)\n\n\nmax_times = max(times)\nmax_hc = max(hc_log[time_col_name])\n\nif max_times > max_hc:\n    for t in times:\n        if t > (max_hc):\n            times.remove(t)    \nelse:\n    hc_log = hc_log[hc_log.Time <= max_times+3]            \n\nhc_log = hc_log.reset_index(drop=True)  # reindex dataframe so that it starts from 0 after having removed rows in it \n'

In [13]:
def insert_row(i, df, line): # insert a row at a given index i and re-index all rows below it
    dfA = df.iloc[:i, ]
    dfB = df.iloc[i:, ]
    df = dfA.append(line).append(dfB).reset_index(drop = True)
    return df

times = spectrum_times
times.sort(reverse = True)
for t in times:
    i=0
    if t < log_times[0]:
        line = pd.DataFrame([t], columns = [time_col_name])
        hc_log = insert_row(i, hc_log, line)
    else:
        while hc_log.get_value(i, col=time_col_name) <= t:
            i += 1   
        line = pd.DataFrame([t], columns = [time_col_name])
        hc_log = insert_row(i, hc_log, line)

In [14]:
hc_list = []

for i in range(hc_log.shape[0]-1):
    value = hc_log.get_value(i, 'h/c')
    if value != 'c' and value != 'h':
        hc_list.append(hc_log.iloc[i+1]['h/c'])

In [15]:
times.sort()

hc_and_times = pd.DataFrame(
    {'Spectrum': spectra_times,
     'hc': hc_list
    })

hc_and_times

NameError: name 'spectra_times' is not defined

In [16]:
# Read log temperature file
def read_log_file_temperature(directory, filename):
    lines = open(os.path.join(directory, filename)).readlines()
    time = [(datetime.datetime.strptime(i[0:17], '%Y%m%d,%H:%M:%S')) for i in lines]
    time_sec = [time[i].second+time[i].minute*60+time[i].hour*3600 for i in range(len(time))]    
    temp_cold = [float((i[-6:-1])) for i in lines] # temperature of external black body
    temp_hot = [float((i[-12:-7])) for i in lines] # temperature of external black body
    return temp_hot, temp_cold, time, time_sec

In [17]:
temp_hot, temp_cold, time, temp_time = read_log_file_temperature('/Users/micol/Documents/Imperial/Year4/Master_Project/bruker_logs', 'thursday2-20180927113645.log')

In [18]:
temp_log = pd.DataFrame(
    {time_col_name: temp_time,
     'hot': temp_hot,
     'cold': temp_cold
    })
temp_log

,Time,cold,hot
0,45405,22.38,64.83
1,45408,22.38,64.60
2,45410,22.39,64.61
3,45412,22.40,64.64
4,45414,22.40,64.66
5,45417,22.40,64.65
6,45419,22.41,64.68
7,45421,22.42,64.71
8,45423,22.41,64.74
9,45425,22.42,64.72


In [19]:
temps_avg = []
temps_std = []

i=0
for spectrum in times[:-1]:
    times_list = list(filter(lambda t: (t>spectrum and t<times[i+1]) , temp_time))
    index_list = [temp_time.index(item) for item in times_list]
    if hc_list[i] == 'h':
        temps_list = [temp_hot[a] for a in index_list]
    elif hc_list[i] == 'c':
        temps_list = [temp_cold[a] for a in index_list]
    temps_avg.append(np.average(temps_list))
    temps_std.append(np.std(temps_list))
    i+=1

In [20]:
final_frame = pd.DataFrame(
    {'Spectrum_(time_from_midnight)': times[:-1],
     'h_or_c': hc_list[:-1],
     'temp_avg': temps_avg,
     'temp_std': temps_std,
    })

final_frame

,Spectrum_(time_from_midnight),h_or_c,temp_avg,temp_std
0,45419.910,c,22.426667,1.105542e-02
1,45433.383,c,22.448333,6.871843e-03
2,45446.855,c,22.468571,8.329931e-03
3,45460.268,c,22.505556,1.065740e-02
4,45478.496,c,22.538333,1.213352e-02
5,45491.910,c,22.565714,1.049781e-02
6,45505.383,c,22.586667,4.714045e-03
7,45518.795,c,22.613333,9.428090e-03
8,45538.796,h,66.315714,7.027642e-02
9,45552.269,h,66.380000,3.109126e-02


In [ ]:
# try with one raw spectrum for now
def read_prn_raw(directory, filename):
    df = pd.read_table(os.path.join(directory, filename))
    arr = df.values
    wns = []
    raws = []

    for a in arr:
        wns.append(a[0])
        raws.append(a[1])
        
    time = str(filename[12:20]).split('_')    
    
    return time, wns, raws

time1, wnsc1, rawsc1 = read_prn_raw('/Users/micol/Documents/Imperial/Year4/Master_Project/prn_files', '(2018_09_27_14_24_05)_Measurement1.prn')
time2, wnsc2, rawsc2 = read_prn_raw('/Users/micol/Documents/Imperial/Year4/Master_Project/prn_files', '(2018_09_27_14_24_05)_Measurement2.prn')
time3, wnsc3, rawsc3 = read_prn_raw('/Users/micol/Documents/Imperial/Year4/Master_Project/prn_files', '(2018_09_27_14_24_05)_Measurement3.prn')
time4, wnsc4, rawsc4 = read_prn_raw('/Users/micol/Documents/Imperial/Year4/Master_Project/prn_files', '(2018_09_27_14_24_05)_Measurement4.prn')

wnsc1